# Quantum mechanical evolution

In [1]:
%load_ext nb_mypy
%nb_mypy On
    
%matplotlib ipympl

Version 1.0.4


In [2]:
import matplotlib.pyplot as plt
import nlsa.fourier_s1 as f1
import nlsa.fourier_t2 as f2
import nlsa.function_algebra as fun
import nlsa.vector_algebra as vec
import numpy as np

from functools import partial
from ipywidgets import widgets, interactive, interact, IntSlider
from more_itertools import take
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from multiprocess import Pool
from nlsa.abstract_algebra import compose_by, multiply_by
from nlsa.dynamics import circle_rotation, orbit
from nlsa.function_algebra import compose, fmap
from nlsa.matrix_algebra import pure_state
from nptyping import Complex, Double, Int, NDArray, Shape
from scipy.linalg import eig, norm
from typing import Callable, Tuple, TypeVar

In [3]:
I = int
I2 = Tuple[int, int]
R = float
R2 = Tuple[float, float]

X = NDArray[Shape["*"], Double]
X2 = NDArray[Shape["*, 2"], Double]
Y = NDArray[Shape["*"], Double]
F = Callable[[X], Y]
F2 = Callable[[X2], Y]

N = TypeVar("N")
V = TypeVar("V", NDArray[Shape["N"], Double], NDArray[Shape["N"], Complex])
W = TypeVar("W",
            NDArray[Shape["*, N"], Double],
            NDArray[Shape["*, N"], Complex])
M = TypeVar("M",
            NDArray[Shape["N, N"], Double],
            NDArray[Shape["N, N"], Complex])

In [4]:
def eig_sorted(a: M) -> Tuple[V, M]:
    w, v = eig(a)
    idx = w.real.argsort()
    w = w[idx]
    v = v[:, idx]
    return w, v

## Von Mises density on the circle

In [5]:
def vm(kappa: R) -> F:
    """Von Mises density centered at pi."""
    f: F = f1.von_mises(kappa, loc=np.pi)
    return f


def vm_fourier(kappa: R, l: I) -> V:
    """Fourier coefficients of von Mises density centered at pi."""
    vm_hat = f1.von_mises_fourier(kappa, loc=np.pi)
    f_hat: V = vm_hat(f1.dual_group(l))
    return f_hat


def vm_mult_op(kappa: R, l: I) -> M:
    """Projected multiplication operator induced by von Mises density.""" 
    mult_op = f1.make_mult_op(l)
    f_mult: M = mult_op(vm_fourier(kappa, 2 * l))
    return f_mult

In [6]:
def feature_map(epsilon: R, l: I) -> Callable[[X], W]:
    """Feature map based on von Mises density."""
    kappa = epsilon ** -2
    xi_0 = f1.von_mises_feature_map(kappa, f1.dual_group(l))
    z = np.linalg.norm(xi_0(np.array([0])))
    
    def xi(x: X) -> W:
        return xi_0(x) / z
    return xi

def qm_recon(a: M, xi: Callable[[X], W]) -> Callable[[X], Y]:
    """Reconstructs classical observable (function) from quantum mechanical
    observable and feature map.
    
    """
    def f(x: X) -> Y:
        omega = pure_state(xi(x))
        return omega(a)
    return f

## Ergodic rotation

In [7]:
a_2pi = 1 / np.sqrt(20)
kappa = 10
l = 3
epsilon = 0.05

a = 2 * np.pi * a_2pi
ls = f1.dual_group(l)
mult_op = f1.make_mult_op(l)
synth = partial(fun.synthesis, f1.fourier_basis(ls))
spec = f1.rotation_koopman_eigs(a)

f = vm(kappa)
f_hat = vm_fourier(kappa, l)
f_mul = mult_op(vm_fourier(kappa, 2 * l))

qsynth = partial(qm_recon, f_mul)
xi = feature_map(epsilon, l)

u = compose_by(fun, circle_rotation(a))
u_cl = multiply_by(vec, spec(f1.dual_group(l)))
u_qm = fmap(multiply_by(vec, spec(-f1.dual_group(l))))

In [8]:
n_plt_x = 128
n_iter = 20
n_par = 2
i_fig = 1

f_orb = orbit(f, u)
fn = take(n_iter + 1, f_orb) 

f_hat_orb = orbit(f_hat, u_cl)
f_cl_orb  = map(synth, f_hat_orb)
fn_cl = take(n_iter + 1, f_cl_orb) 

pl = Pool(n_par)
xi_orb = orbit(xi, u_qm)
f_qm_orb  = map(qsynth, xi_orb)
fn_qm = take(n_iter + 1, f_qm_orb) 

x = np.linspace(0, 2*np.pi, n_plt_x)
x1 = x / np.pi

if plt.fignum_exists(i_fig):
    plt.close(i_fig)
    
fig, ax = plt.subplots(num=i_fig)
n_slider = IntSlider(value=0, min=0, max=n_iter, continuous_update=False)

@interact(n=n_slider) 
def plotfunc(n):
    ax.cla()
    fx = fn[n](x)
    f_clx = np.real(fn_cl[n](x))
    f_qmx = np.real(pl.map(fn_qm[n], x))
    ax.plot(x1, fx, label="true")
    ax.plot(x1, f_clx, label="classical")
    ax.plot(x1, f_qmx, label="quantum")
    ax.legend()
    ax.set_xlim(0, 2)
    ax.grid()
    ax.set_xlabel("$\\theta_1/\\pi$")
    ax.set_title(f'Circle rotation by angle $a={a_2pi:.3f}\\times 2\pi$; iteration $n= {n}$')
    plt.show()

interactive(children=(IntSlider(value=0, continuous_update=False, description='n', max=20), Output()), _dom_cl…